In [41]:
%pip install --upgrade --quiet langchain-openai



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [59]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import numpy as np
import json


In [43]:

# Get the current working directory
current_dir = os.getcwd()

# Go up two levels to reach the directory containing keys.env
parent_dir = os.path.dirname(os.path.dirname(current_dir))

# Construct the full path to keys.env
env_path = os.path.join(parent_dir, 'keys.env')

print(f"Looking for keys.env at: {env_path}")

# Check if the file exists
if os.path.exists(env_path):
    print("keys.env file found.")
    # Load environment variables from `keys.env`
    load_dotenv(env_path)

    # Accessing the variables
    bias1_key1 = os.getenv('BIAS1_KEY1')
    azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
    azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
    azure_openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
    azure_openai_chat_deployment_name_gpt3 = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME_GPT3')
    azure_openai_embed_deployment_large = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME_EMBED_LARGE')
    huggingfacehub_api_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')



Looking for keys.env at: /Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/bias_llm_clinical_challenge/keys.env
keys.env file found.


In [44]:
%pip install --upgrade --quiet langchain-openai



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [45]:
# --- with LangChain
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    api_key=azure_openai_api_key,
    azure_deployment=azure_openai_embed_deployment_large,
    openai_api_version=azure_openai_api_version
)


In [61]:
# =========== Helper functions ===========
def get_embedding(text):
    """
    This function retrieves the embedding for a given text using the specified model.
    """
    return embeddings.embed_query(text)

def cosine_similarity(a, b):
    """
    This function calculates the cosine similarity between two embedding vectors.
    """
    if a is None or b is None:
        return None
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def calculate_similarity(text1, text2):
    embedding1 = get_embedding(text1)
    embedding2 = get_embedding(text2)
    
    if embedding1 is None or embedding2 is None:
        print("Error: Unable to generate embeddings for one or both texts.")
        return None
    
    return cosine_similarity(embedding1, embedding2)

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def truncate_text(text: str, max_tokens: int = 8192, encoding_name: str = "cl100k_base") -> str:
    """Truncates the text to the specified maximum number of tokens."""
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    if len(tokens) > max_tokens:
        truncated_tokens = tokens[:max_tokens]
        truncated_text = encoding.decode(truncated_tokens)
        warnings.warn(f"Text truncated from {len(tokens)} to {max_tokens} tokens.")
        return truncated_text
    return text


In [47]:
# Example usage
text1 = "this is an awesome test document"
text2 = "this is an aweful test document"
query_result = calculate_similarity(text1, text2)

if query_result is not None:
    print(f"Cosine similarity: {query_result}")
else:
    print("Failed to calculate similarity.")
    

Cosine similarity: 0.8479392455403287


In [48]:
embed_test=get_embedding("Testing")


In [49]:
type(embed_test)


list

# Experiment 1

## 1a Gender original

In [50]:
df_path=os.path.join(parent_dir,'results/experiment1/gender/cleaned_data_other_columns.csv')
df=pd.read_csv(df_path)
df.head()


,test_image,link,opb_shuffled,key_id,opc_shuffled,clinical_question,pregnancy,age_group,answer_idx_shuffled,woman_health,...,opc,opb,diagnosis,experiment,opa_shuffled,explanation,augmented_gender,original_gender,opd_shuffled,answer_idx
0,1,https://jamanetwork.com/journals/jamaoncology/...,Kimura disease,4_G_o_o_m,Classic Hodgkin lymphoma,what is your diagnosis?,0,31-40,d,0,...,T-cell acute lymphoblastic lymphoma/leukemia,Classic Hodgkin lymphoma,D. Myeloid/lymphoid neoplasms with eosinophili...,G,T-cell acute lymphoblastic lymphoma/leukemia,The differential diagnoses in young men with e...,original,male,Myeloid/lymphoid neoplasms with eosinophilia a...,D
1,1,https://jamanetwork.com/journals/jamaoncology/...,Kimura disease,4_G_a_f_m,Classic Hodgkin lymphoma,what is your diagnosis?,0,31-40,d,0,...,T-cell acute lymphoblastic lymphoma/leukemia,Classic Hodgkin lymphoma,D. Myeloid/lymphoid neoplasms with eosinophili...,G,T-cell acute lymphoblastic lymphoma/leukemia,The differential diagnoses in young men with e...,female,male,Myeloid/lymphoid neoplasms with eosinophilia a...,D
2,1,https://jamanetwork.com/journals/jamaoncology/...,Kimura disease,4_G_a_n_m,Classic Hodgkin lymphoma,what is your diagnosis?,0,31-40,d,0,...,T-cell acute lymphoblastic lymphoma/leukemia,Classic Hodgkin lymphoma,D. Myeloid/lymphoid neoplasms with eosinophili...,G,T-cell acute lymphoblastic lymphoma/leukemia,The differential diagnoses in young men with e...,neutral,male,Myeloid/lymphoid neoplasms with eosinophilia a...,D
3,0,https://jamanetwork.com/journals/jamaoncology/...,Medication adverse effect,11_G_o_o_f,Melanoma,what is your diagnosis?,0,51-60,b,0,...,Medication adverse effect,Melanoma,C. Medication adverse effect,G,Laugier-Hunziker syndrome,Oral hyperpigmentation is a rare adverse effec...,original,female,Oral involvement of mycosis fungoides,C
4,0,https://jamanetwork.com/journals/jamaoncology/...,Medication adverse effect,11_G_a_m_f,Melanoma,what is your diagnosis?,0,51-60,b,0,...,Medication adverse effect,Melanoma,C. Medication adverse effect,G,Laugier-Hunziker syndrome,Oral hyperpigmentation is a rare adverse effec...,male,female,Oral involvement of mycosis fungoides,C


In [51]:
print(f"Columns: {df.columns}")
print(f"Shape: {df.shape}")


Columns: Index(['test_image', 'link', 'opb_shuffled', 'key_id', 'opc_shuffled',
       'clinical_question', 'pregnancy', 'age_group', 'answer_idx_shuffled',
       'woman_health', 'gender', 'version_gender', 'version_binary', 'case_id',
       'field', 'figure', 'opd', 'test_lab', 'test_other', 'ethnicity',
       'version', 'age', 'normalized_question', 'case', 'opa', 'gender_int',
       'question', 'answer', 'opc', 'opb', 'diagnosis', 'experiment',
       'opa_shuffled', 'explanation', 'augmented_gender', 'original_gender',
       'opd_shuffled', 'answer_idx'],
      dtype='object')
Shape: (600, 38)


In [52]:
df['case_id'].nunique()


200

#### 1a.a. Explanation -> per unique case_id

In [53]:
import pandas as pd
import numpy as np
import os
import json

def process_dataframe_per_caseid(df, get_embedding, cosine_similarity, max_tokens=8192):
    # Create df_unique_caseid with the first row for each unique case_id
    df_unique_caseid = df.groupby('case_id').first().reset_index()
    
    # Initialize new columns
    new_columns = ['explanation_truncated', 'explanation_tokens', 'explanation_embedding']
    response_columns = [col for col in df_unique_caseid.columns if 'response' in col]
    for col in response_columns:
        new_columns.extend([f'{col}_truncated', f'{col}_tokens', f'{col}_embedding', f'{col}_embedding_sim'])
    
    for col in new_columns:
        if col not in df_unique_caseid.columns:
            df_unique_caseid[col] = None
    
    # Process df_unique_caseid
    total_rows = len(df_unique_caseid)
    for i, (index, row) in enumerate(df_unique_caseid.iterrows(), 1):
        # Create truncated versions if necessary
        df_unique_caseid.at[index, 'explanation_truncated'] = truncate_text(row['explanation'], max_tokens)
        df_unique_caseid.at[index, 'explanation_tokens'] = num_tokens_from_string(row['explanation'])
        
        # Calculate embedding for explanation
        explanation_embedding = get_embedding(df_unique_caseid.at[index, 'explanation_truncated'])
        df_unique_caseid.at[index, 'explanation_embedding'] = json.dumps(explanation_embedding)
        
        # Process response columns
        for col in response_columns:
            df_unique_caseid.at[index, f'{col}_truncated'] = truncate_text(row[col], max_tokens)
            df_unique_caseid.at[index, f'{col}_tokens'] = num_tokens_from_string(row[col])
            response_embedding = get_embedding(df_unique_caseid.at[index, f'{col}_truncated'])
            df_unique_caseid.at[index, f'{col}_embedding'] = json.dumps(response_embedding)
            df_unique_caseid.at[index, f'{col}_embedding_sim'] = cosine_similarity(
                explanation_embedding,
                response_embedding
            )
        
        # Print progress and save to CSV every 10%
        if i % (total_rows // 10) == 0 or i == total_rows:
            print(f"Processed {i}/{total_rows} rows ({i/total_rows*100:.2f}%)")
            df_unique_caseid.to_csv(f'df_unique_caseid_progress_{i}.csv', index=False)

    # Process the original dataframe
    df_result = df.copy()
    
    # Initialize new columns in df_result
    for col in new_columns:
        if col not in df_result.columns:
            df_result[col] = None
    
    # Map new columns from df_unique_caseid to df_result
    columns_to_map = ['explanation_embedding'] + new_columns
    for col in columns_to_map:
        df_result[col] = df_result['case_id'].map(dict(zip(df_unique_caseid['case_id'], df_unique_caseid[col])))
    
    # Save the final dataframe
    df_results=df[['case_id','explanation','explanation_truncated','explanation_tokens','explanation_embedding']]
    df_result.to_csv('df_G_unique_caseid.csv', index=False)
    
    return df_result



In [54]:
result_df = process_dataframe_per_caseid(df, get_embedding, cosine_similarity)


NameError: name 'tiktoken' is not defined

In [ ]:
df_results.columns


Index(['test_image', 'link', 'opb_shuffled', 'key_id', 'opc_shuffled',
       'clinical_question', 'pregnancy', 'age_group', 'answer_idx_shuffled',
       'woman_health', 'gender', 'version_gender', 'version_binary', 'case_id',
       'field', 'figure', 'opd', 'test_lab', 'test_other', 'ethnicity',
       'version', 'age', 'normalized_question', 'case', 'opa', 'gender_int',
       'question', 'answer', 'opc', 'opb', 'diagnosis', 'experiment',
       'opa_shuffled', 'explanation', 'augmented_gender', 'original_gender',
       'opd_shuffled', 'answer_idx', 'explanation_truncated',
       'explanation_tokens', 'explanation_embedding'],
      dtype='object')

In [ ]:
df_results=pd.read_csv('df_G_unique_caseid.csv')
df_results=df_results[['case_id','explanation','explanation_truncated','explanation_tokens','explanation_embedding']]
df_results.to_csv('df_G_unique_caseid.csv', index=False)


In [51]:
df_results.isna().sum() 


case_id                  0
explanation              0
explanation_truncated    0
explanation_tokens       0
explanation_embedding    0
dtype: int64

### 1a.b. LLM output --> Per llm

In [55]:
df_G_unique_caseid=pd.read_csv('df_G_unique_caseid.csv')
print(f"Columns: {df_G_unique_caseid.columns}")


Columns: Index(['case_id', 'explanation', 'explanation_truncated', 'explanation_tokens',
       'explanation_embedding'],
      dtype='object')


In [56]:
import os
import pandas as pd

gender_folder = os.path.join(parent_dir, 'results/experiment1/gender')

dfs = []
for filename in os.listdir(gender_folder):
    if filename.startswith('result_G_') and filename.endswith('.csv'):
        filepath = os.path.join(gender_folder, filename)
        df = pd.read_csv(filepath)
        # Extract LLM name from filename (assuming it's part of the filename)
        llm_name = filename.split('_')[2].split('.')[0]  # Adjust this based on your actual filename format
        df['llm_name'] = llm_name
        dfs.append(df)

# Combine all DataFrames
df_combined = pd.concat(dfs, ignore_index=True)

# Create the final DataFrame
df_llm = df_combined[['case_id', 'explanation','key_id']].drop_duplicates()

for llm_name in df_combined['llm_name'].unique():
    print(f"Processing {llm_name}")
    columns_to_keep = [col for col in df_combined.columns if f'llm_{llm_name}_response' in col]
    df_llm_subset = df_combined[df_combined['llm_name'] == llm_name][columns_to_keep]
    df_llm = pd.merge(df_llm, df_llm_subset, left_index=True, right_index=True, how='left')

print(df_llm.shape)
print(df_llm.columns)

df_llm.to_csv('df_G_llms.csv', index=False)


Processing gpt4omini
Processing gpt3
Processing gpt4turbo
Processing gpt4o
(600, 11)
Index(['case_id', 'explanation', 'key_id', 'llm_gpt4omini_response2a',
       'llm_gpt4omini_response1', 'llm_gpt3_response1', 'llm_gpt3_response2a',
       'llm_gpt4turbo_response2a', 'llm_gpt4turbo_response1',
       'llm_gpt4o_response1', 'llm_gpt4o_response2a'],
      dtype='object')


In [57]:
import pandas as pd
import numpy as np

def add_explanation_embedding(df_llm, df_G_unique_caseid):
    # Ensure 'case_id' is the index in df_G_unique_caseid
    if 'case_id' in df_G_unique_caseid.columns:
        df_G_unique_caseid = df_G_unique_caseid.set_index('case_id')

    # Convert string representation of list to actual list
    df_G_unique_caseid['explanation_embedding'] = df_G_unique_caseid['explanation_embedding'].apply(eval)

    # Merge df_llm with df_G_unique_caseid
    df_llm = df_llm.merge(df_G_unique_caseid['explanation_embedding'], 
                          left_on='case_id', 
                          right_index=True, 
                          how='left')

    # Convert the explanation_embedding to a numpy array
    df_llm['explanation_embedding'] = df_llm['explanation_embedding'].apply(np.array)

    return df_llm

# Usage
df_llm = add_explanation_embedding(df_llm, df_G_unique_caseid)

# Verify the result
print(df_llm.shape)
print(df_llm.columns)


(1800, 12)
Index(['case_id', 'explanation', 'key_id', 'llm_gpt4omini_response2a',
       'llm_gpt4omini_response1', 'llm_gpt3_response1', 'llm_gpt3_response2a',
       'llm_gpt4turbo_response2a', 'llm_gpt4turbo_response1',
       'llm_gpt4o_response1', 'llm_gpt4o_response2a', 'explanation_embedding'],
      dtype='object')


In [60]:
type(df_llm['explanation_embedding'].iloc[0])


numpy.ndarray

In [66]:
import pandas as pd
import numpy as np
import json
import warnings

def process_response_columns(df_llm, max_tokens=8192):
    # Identify response columns
    response_columns = [col for col in df_llm.columns if 'response' in col]
    
    # Initialize new columns
    new_columns = []
    for col in response_columns:
        new_columns.extend([f'{col}_truncated', f'{col}_tokens', f'{col}_embedding', f'{col}_similarity'])
    
    for col in new_columns:
        if col not in df_llm.columns:
            df_llm[col] = None
    
    # Process df_llm
    total_rows = len(df_llm)
    for i, (index, row) in enumerate(df_llm.iterrows(), 1):
        # Process explanation embedding if it's not already processed
        if isinstance(row['explanation_embedding'], str):
            explanation_embedding = np.array(json.loads(row['explanation_embedding']))
        else:
            explanation_embedding = row['explanation_embedding']
        
        # Process response columns
        for col in response_columns:
            # Ensure the response is a string
            response_text = str(row[col])
            
            # Create truncated versions if necessary
            df_llm.at[index, f'{col}_truncated'] = truncate_text(response_text, max_tokens)
            df_llm.at[index, f'{col}_tokens'] = num_tokens_from_string(response_text)
            
            # Calculate embedding for response
            truncated_response = df_llm.at[index, f'{col}_truncated']
            response_embedding = get_embedding(truncated_response)
            
            # Store the embedding as JSON string
            df_llm.at[index, f'{col}_embedding'] = json.dumps(response_embedding)
            
            # Calculate similarity
            df_llm.at[index, f'{col}_similarity'] = cosine_similarity(explanation_embedding, response_embedding)
        
        # Print progress and save to CSV every 10%
        if i % (total_rows // 10) == 0 or i == total_rows:
            print(f"Processed {i}/{total_rows} rows ({i/total_rows*100:.2f}%)")
            df_llm.to_csv(f'df_G_llms_embeddings_progress_{i}.csv', index=False)

    # Save the final dataframe
    df_llm.to_csv('df_G_llms_embeddings.csv', index=False)
    
    return df_llm

# Usage
df_llm = process_response_columns(df_llm)

# Verify the result
print(df_llm.shape)
print(df_llm.columns)

# Print first few values of a new embedding and similarity column
response_col = next(col for col in df_llm.columns if 'response' in col)
print(f"\nFirst few values of {response_col}_embedding:")
print(df_llm[f"{response_col}_embedding"].iloc[0][:100])  # Print first 100 characters
print(f"\nFirst few values of {response_col}_similarity:")
print(df_llm[f"{response_col}_similarity"].head())


/var/folders/s2/jdv7kxv13hgf_3qqk7bg5p3r0000gn/T/ipykernel_8881/4032418722.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_llm[col] = None
/var/folders/s2/jdv7kxv13hgf_3qqk7bg5p3r0000gn/T/ipykernel_8881/4032418722.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_llm[col] = None
/var/folders/s2/jdv7kxv13hgf_3qqk7bg5p3r0000gn/T/ipykernel_8881/4032418722.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider jo

TypeError: argument 'text': 'Series' object cannot be converted to 'PyString'